In [126]:
import nibabel as nib
import numpy as np
from scipy.ndimage.interpolation import zoom
import matplotlib.pyplot as plt
from skimage import transform

In [10]:
def get_roi(mask_path):

    img_nii = nib.load(mask_path)
    img = img_nii.get_data()
    roi = np.zeros(img.shape, dtype = np.uint8)
    x_list, y_list, z_list = [], [], []
    for i in range(img.shape[0]):
        if np.sum(img[i, :, :]) > 20:
            x_list.append(i)
    for i in range(img.shape[1]):
        if np.sum(img[:, i, :]) > 20:
            y_list.append(i)
    for i in range(img.shape[2]):
        if np.sum(img[:, :, i]) > 20:
            z_list.append(i)
            #roi[:, :, i] = 1
    x_cent, y_cent, z_cent =  (x_list[0] + x_list[-1]) / 2, (y_list[0] + y_list[-1]) / 2, (z_list[0] + z_list[-1]) / 2
    print ('the center is: ', x_cent, y_cent, z_cent)
    
    
    return x_cent, y_cent, z_cent

In [107]:
img_path = '/nfs/masi/MCL/nifti/combine/25176612842/20110330/25176612842time20110330.nii.gz'
mask_path = '/nfs/masi/xuk9/MCL/thorax_reg_roi_full/lung_mask/25176612842time20110330.nii.gz'
img_nii = nib.load(img_path)
print (img_nii.shape)

(512, 512, 62)


In [108]:
mask_nii = nib.load(mask_path)
resol = [mask_nii.header['pixdim'][1], mask_nii.header['pixdim'][2], mask_nii.header['pixdim'][3]]
resol_img = [img_nii.header['pixdim'][1], img_nii.header['pixdim'][2], img_nii.header['pixdim'][3]]
print (resol, resol_img)

[0.5625, 0.5625, 5.0] [0.5625, 0.5625, 5.0]


In [110]:
center = get_roi(mask_path)

the center is:  258.0 264.0 31.0


In [111]:
lens =  [200 * 1.5 / resol, 200 * 1.5 / resol, 200 * 1.6 / resol]  #
x_begin, x_end = center[0] - lens[0] / 2, center[0] + lens[0] / 2
y_begin, y_end = center[1] - lens[1] / 2, center[1] + lens[1] / 2
z_begin, z_end = center[2] - lens[2] / 2, center[2] + lens[2] / 2

In [119]:
img_new = img_nii.get_data() #[x_begin: x_end, y_begin: y_end, z_begin: z_end] 
x_begin, x_end, y_begin, y_end, z_begin, z_end = max(0,int(x_begin)), int(x_end), max(0, int(y_begin)), int(y_end), max(0,int(z_begin)), int(z_end)
img_new = img_new[x_begin: x_end, y_begin: y_end, z_begin: z_end]

In [121]:
resize_factor = np.array([200, 200, 200]) / np.array([int(x_end)- int(x_begin), int(y_end) - int(y_begin), int(z_end) - int(z_begin)])
print (resize_factor)

[0.38167939 0.37735849 3.33333333]


In [130]:
img_save = transform.resize(img_new, [200, 200, 200], mode='edge', preserve_range='True')
#zoom(img_new, resize_factor, mode = 'nearest',order=3)

In [128]:
y_begin, y_end, img_new.shape, img_save.shape

(0, 530, (512, 512, 60), (200, 200, 200))

In [131]:
save_nii = nib.Nifti1Image(img_save, np.eye(4))
nib.save(save_nii, '/nfs/masi/gaor2/tmp/resample2.nii.gz')

In [140]:
img_nii = nib.load('/nfs/masi/gaor2/data/MCL/resample_roi/25053962798time20080101.nii.gz')
img = img_nii.get_data()


In [149]:
np.max(img)

1970.6856504818024

In [152]:
img_new = img
img_new[img_new < -1000] = -1000
img_new[img_new > 1000] = 1000
img_new = img_new.astype(np.int16)

In [153]:
save_nii = nib.Nifti1Image(img_new, np.eye(4))
nib.save(save_nii, '/nfs/masi/gaor2/tmp/resample.nii.gz')